In [70]:
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
import pandas as pd
import pandapower as pp
import re
import numpy as np

root_dir = "C:/Users/Damian/OneDrive/Share/Penn/Research/OPF/"
data_dir = root_dir + "data/denmark/"
    

In [71]:
def fix_decimals(value):
    # replace things like 10,00 with 10.00 - European to US decimals
    if isinstance(value, str) and re.search("[0-9]+,[0-9]+", value):
        return float(value.replace(',', '.'))
    return value

bus_data = pd.read_excel(data_dir + "raw.xlsx", skiprows=range(0, 3), sheet_name='Bus').sort_values('Bus Index').applymap(
    fix_decimals)
line_data = pd.read_excel(data_dir + "raw.xlsx", skiprows=range(0, 3), sheet_name='Line').applymap(fix_decimals)
trafo2_data = pd.read_excel(data_dir + "raw.xlsx", skiprows=range(0, 3), sheet_name='Transformer2').applymap(fix_decimals)
trafo3_data = pd.read_excel(data_dir + "raw.xlsx", skiprows=range(0, 3), sheet_name='Transformer3').applymap(fix_decimals)
generator_data = pd.read_excel(data_dir + "raw.xlsx", skiprows=range(0, 3), sheet_name='Generator').applymap(fix_decimals)
load_data = pd.read_excel(data_dir + "raw.xlsx", skiprows=range(0, 3), sheet_name='Load').applymap(fix_decimals)
shunt_data = pd.read_excel(data_dir + "raw.xlsx", skiprows=range(0, 3), sheet_name='Shunt').applymap(fix_decimals)
hvdc_data = pd.read_excel(data_dir + "raw.xlsx", skiprows=range(0, 3), sheet_name='HVDC').applymap(fix_decimals)


In [97]:
net = pp.create_empty_network("Denmark")
net.sn_mva = 1.0

for _, row in bus_data.iterrows():
    row: pd.Series = row
    voltage = row['Voltage base[kV]']
    pp.create_bus(net, vn_kv=voltage, name=row['Bus Name'], index=row['Bus Index'], 
                  max_vm_pu=row["Voltage max[pu]"], min_vm_pu=row["Voltage min[pu]"])

for _, row in line_data.iterrows():
    row: pd.Series = row

    type = row['Line type']
    L: float = row['Length[km]']
    R: float = max(1, row['R1[Ohm]']) # TODO: remove
    X: float = row['X1[Ohm]']
    G: float = row['G1[uS]']
    B: float = row['B1[uS]']
    limit_current = row['Nominal Current[kA]'] * 10  # TODO: Verify
    C = 10  # TODO: verify
    if type == "Null Line" or (R < 0.01 or X < 0.01):
        pp.create_switch(net, row['Node 1'], row['Node 2'], 'b', True, z_ohm=R)
    elif type == "Overhead Line" or type == "Cable":
        pp.create_line_from_parameters(net, row['Node 1'], row['Node 2'], L, R/L, X/L, C, limit_current, 
                                       g_us_per_km=G/L, max_loading_percent=100)
    
    elif type == "Equivalent Impedance":
        v_base: float = row['Nominal Voltage[kV]']
        i_base: float = row['Nominal Current[kA]']
        base_z: float = v_base / i_base
        sn_mva: float = np.sqrt(3) * v_base * i_base
        pp.create_impedance(net, row['Node 1'], row['Node 2'], R/base_z, X/base_z, sn_mva, 
                            R/base_z, X/base_z, in_service=True)
    else:
        raise RuntimeError("{} is an unknown line type".format(type))
        
# Based on https://www.fs.fed.us/database/acad/elec/greenbook/10_shortcalc.pdf
# X/R is ~3.5 therefore %R is %Z/(1+3.5)
Z_XR_factor = 1/(1+3.5)
for _, row in trafo2_data.iterrows():
    row: pd.Series = row
    # Assuming core losses are approximately equal to no load loss
    # Assuming short circuit voltage is purely real
    # Assuming max loading percent is 100
    pp.create_transformer_from_parameters(net, row['High.V Bus Index'], row['Low.V Bus Index'], row['Sn[MVA]'],
                                          row['Un.H[kV]'], row['Un.L[kV]'], row['uk[%]']*Z_XR_factor, row['uk[%]'], 
                                          row['No Load Losses[kW]'], row['No Load Current[%]'], in_service=True,
                                          tap_min=row['Min.Tap'], tap_max=row['Max.Tap'], tap_neutral=row['Neu.Tap'],
                                          shift_degree=row['dUphase[deg]'], tap_step_percent=row['dU.Tap[%]'],
                                          tap_side="hv", tap_pos=row['Act.Tap'], max_loading_percent=100,
                                          name=row['2-Transformer Name'])

for _, row in trafo3_data.iterrows():
    row: pd.Series = row
    # Same assumptions as trafo2, AND
    # Assuming phase shift is the same for LV and MV
    control_bus = row['Control Bus Index']
    hBusIndex = row['High.V Bus Index']
    mBusIndex = row['Mid.V Bus Index']
    lBusIndex = row['Low.V Bus Index']
    tap_side = row['Tap Side'].lower()
    shift_degree = row['dUphase.Tap[deg]']

    # Same as trafo2
    pp.create_transformer3w_from_parameters(net, hBusIndex, mBusIndex, lBusIndex,
                                            row['Un.H[kV]'], row['Un.M[kV]'], row['Un.L[kV]'],
                                            row['Sn[MVA]'], row['Sn[MVA]'], row['Sn[MVA]'],
                                            row['uk.HV_MV[%]'], row['uk.MV_LV[%]'], row['uk.LV_HV[%]'],
                                            row['uk.HV_MV[%]']*Z_XR_factor,
                                            row['uk.MV_LV[%]']*Z_XR_factor, 
                                            row['uk.LV_HV[%]']*Z_XR_factor,
                                            row['No Load Losses[kW]'], row['No Load Current[%]'], tap_side=tap_side,
                                            tap_min=row['Min.Tap'], tap_max=row['Max.Tap'], tap_neutral=row['Neu.Tap'],
                                            tap_step_percent=row['dU.Tap[%]'], tap_pos=row['Act.Tap'],
                                            in_service=True, shift_lv_degree=shift_degree, shift_mv_degree=shift_degree,
                                            name=row['3_Transformer Name'], max_loading_percent=100)

for _, row in generator_data.iterrows():
    row: pd.Series = row

    control_type = row['Control Type']
    slack: bool = control_type == "SL"  # this is a slack generator
    p_mw = row['Act.P[MW]']
    q_mvar = row['Act.Q[Mvar]']
    if control_type == "PQ":
        id = pp.create_sgen(net, row['Bus Index'], p_mw, q_mvar,
                       row['Nominal Apparent Power[MVA]'], row['Generator Name'],
                       max_p_mw=row['Pmax[MW]']*10, min_p_mw=row["Pmin[MW]"],
                       max_q_mvar=row['Qmax[Mvar]']*10, min_q_mvar=row['Qmin[Mvar]'],
                       controllable=True)
    elif control_type == "PV" or slack:

        # Approximating these plants which compensate for voltage variance of renewable plants by
        # placing them on the Control bus
        id = pp.create_gen(net, row['Bus Index'], p_mw, row['Uref[pu]'],
                      row['Nominal Apparent Power[MVA]'], row['Generator Name'],
                      max_p_mw=row['Pmax[MW]'], min_p_mw=row['Pmin[MW]']*0,
                      max_q_mvar=row['Qmax[Mvar]']*10, min_q_mvar=row['Qmin[Mvar]'],
                      controllable=True, slack=slack)
    else:
        raise Exception("{} control type is not defined.".format(control_type))

for _, row in load_data.iterrows():
    row: pd.Series = row

    p_mw: float = row['Act.P[MW]']
    q_mvar: float = row['Act.Q[Mvar]']
    # ZIP model data: https://www.diva-portal.org/smash/get/diva2:1085518/FULLTEXT01.pdf page 61 (83 on pdf)
    # [Z I P]
    p: np.ndarray = np.array([0.38, 0.41, 0.21])
    q: np.ndarray = np.array([1, 0, 0])
    aggregate: np.ndarray = (p * p_mw + q * q_mvar)

    if np.sum(aggregate) == 0:
        aggregate = np.array([100, 0, 0])
    else:
        aggregate = 100 * aggregate / np.sum(aggregate)  # values should be in [0,100]
    aggregate = aggregate.astype(int)
        
    if not (row['Area Name'] == 'ENDKW' or row['Area Name'] == 'ENDKE'):
        id = pp.create_ext_grid(net, row['Load Index'], name=row['Load Name'],
                                max_p_mw=700, min_p_mw=-700, max_q_mvar=700, min_q_mvar=-700)
    else:
        assert p_mw >= 0
        pp.create_load(net, row['Load Index'], p_mw, q_mvar, aggregate[0], aggregate[1], controllable=False)

for _, row in shunt_data.iterrows():
    row: pd.Series = row
    switchable = row['Switchable']
    step = row['Act.Step']
    pp.create_shunt(net, row['Bus Index'], row['L[Mvar]'] + row['C[Mvar]'], vn_kv=row['Nominal Voltage[kV]'],
                    step=step, max_step=row['Max.Step'], name=row['Shunt Name'])

for _, row in hvdc_data.iterrows():
    row: pd.Series = row
    
    min_p_mw = 0 if row['Pmax[MW]'] > 0 else row['Pmax[MW]']
    max_p_mw = row['Pmax[MW]'] if row['Pmax[MW]'] > 0 else 0
    assert max_p_mw > min_p_mw
    
    min_q_mvar = 0 if row['Q[Mvar]'] > 0 else row['Q[Mvar]']
    max_q_mvar = row['Q[Mvar]'] if row['Q[Mvar]'] > 0 else 0
    assert max_q_mvar > min_q_mvar

    pp.create_ext_grid(net, row['Bus Index'], name=row['HVDC Name'], 
                       min_p_mw=min_p_mw, max_p_mw=max_p_mw, min_q_mvar=min_q_mvar, max_q_mvar=max_q_mvar)

In [98]:
import networkx as nx
from pandapower import topology, converter
from pandapower.plotting import simple_plot, to_html
from pandapower.plotting.plotly import simple_plotly, vlevel_plotly
from matplotlib import pyplot as plt

def add_cost(net):
    for id, row in net['sgen'].iterrows():
        pp.create_poly_cost(net, id, 'sgen', cp1_eur_per_mw=1)
    for id, row in net['gen'].iterrows():
        pp.create_poly_cost(net, id, 'gen', cp1_eur_per_mw=1)
    for id, row in net['ext_grid'].iterrows():
        pp.create_poly_cost(net, id, 'ext_grid', cp1_eur_per_mw=1)

nx_graph = pp.topology.create_nxgraph(net)
CCs = sorted(list(nx.connected_components(nx_graph)), key=len, reverse=True)

net_train = pp.select_subnet(net, list(CCs[0]))
net_test = pp.select_subnet(net, list(CCs[1]))
add_cost(net_train)
add_cost(net_test)

pp.to_json(net_train, data_dir + "train.json")
pp.to_json(net_test, data_dir + "test.json")
pp.plotting.to_html(net_train, data_dir + "train.html")
pp.plotting.to_html(net_test, data_dir + "test.html")

In [99]:
if net_train is None:
    net_train = pp.from_json(data_dir + "train.json")
    net_test = pp.from_json(data_dir + "test.json")

In [100]:
pp.runpp(net_train, max_iteration=10)
pp.plotting.to_html(net_train, data_dir + "train.html")
#pp.converter.to_mpc(net_train, data_dir + "train.mat", mode="opf")
#pp.runopp(net_train, calculate_voltage_angles=False, verbose=True)
#pp.lf_info(net_train)

def check_constraints(net):
    gen_types = ['gen', 'sgen']
    for type in gen_types:
        print("Type %s" % type)
        for (id, gen), (_, res) in zip(net_train[type].iterrows(), net_train['res_%s'%type].iterrows()):
            p_mw = res['p_mw']
            q_mvar = res['q_mvar']
            if not gen['min_p_mw'] <= p_mw <= gen['max_p_mw']:
                print("At %d, voilated P[MW] %d <= %d <= %d" % (id, gen['min_p_mw'], p_mw, gen['max_p_mw']))
            if not gen['min_q_mvar'] <= q_mvar <= gen['max_q_mvar']:
                print("At %d, voilated Q[Mvar] %d <= %d <= %d" % (id, gen['min_q_mvar'], q_mvar, gen['max_q_mvar']))
            
check_constraints(net_train)

Type gen
Type sgen


In [110]:
from julia.PowerModels import run_ac_opf
pp.runpm_ac_opf(net_train)
pp.plotting.to_html(net_train, data_dir + "iceland_train_opf.html")

This pandapower network includes the following parameter tables:
   - bus (189 elements)
   - load (51 elements)
   - gen (34 elements)
   - shunt (4 elements)
   - ext_grid (1 element)
   - line (91 elements)
   - trafo (115 elements)
   - poly_cost (35 elements)
   - bus_geodata (189 elements)
 and the following results tables:
   - res_bus (189 elements)
   - res_line (91 elements)
   - res_trafo (115 elements)
   - res_ext_grid (1 element)
   - res_load (51 elements)
   - res_shunt (4 elements)
   - res_gen (34 elements)


In [111]:
print(net)

This pandapower network includes the following parameter tables:
   - bus (189 elements)
   - load (51 elements)
   - gen (34 elements)
   - shunt (4 elements)
   - ext_grid (1 element)
   - line (91 elements)
   - trafo (115 elements)
   - poly_cost (35 elements)
   - bus_geodata (189 elements)
 and the following results tables:
   - res_bus (189 elements)
   - res_line (91 elements)
   - res_trafo (115 elements)
   - res_ext_grid (1 element)
   - res_load (51 elements)
   - res_shunt (4 elements)
   - res_gen (34 elements)


In [8]:
pp.diagnostic(net_train, 'detailed', warnings_only=True, return_result_dict=False)



_____________ PANDAPOWER DIAGNOSTIC TOOL _____________ 

_____________ END OF PANDAPOWER DIAGNOSTIC _____________ 


In [9]:
simple_plot(net_train)
plt.savefig(data_dir + "train.png")
simple_plot(net_test)
plt.savefig(data_dir + "test.png")

No or insufficient geodata available --> Creating artificial coordinates. This may take some time


TypeError: can't pickle PyCall.jlwrap objects

In [4]:
import pandapower as pp
import pandapower.networks
net = pp.networks.iceland()
pp.runpp(net)